In [2]:
import requests
import random
import pandas as pd
import time
from tqdm import tqdm
from my_utils import *

In [3]:
db_open()
sql_execute('select * from lol_min_datas')
db_close()

Execution failed on sql 'select * from lol_min_datas': ORA-00942: 테이블 또는 뷰가 존재하지 않습니다
오라클 정상적으로 닫힘


C:\Users\user\게임아이 수업자료\my_utils.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=db)


In [4]:
riot_api_key='RGAPI-4445a220-fdde-47a7-a07f-258499614db7'

In [5]:
def get_puu_id(summoner_id,riot_api_key):
    url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={riot_api_key}'
    res = requests.get(url).json()
    puu_id = res['puuid']
    return puu_id

In [6]:
def get_match_ids(puu_id,riot_api_key,page=20):
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?type=ranked&start=0&count={page}&api_key={riot_api_key}'
    res = requests.get(url).json()
    return res

In [7]:
# 미니프로젝트꺼랑 다름
def get_matches_timelines(match_ids,riot_api_key):
    match_info_list=[]
    for match_id in match_ids:
        url1 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={riot_api_key}' 
        url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={riot_api_key}'
        res1 = requests.get(url1).json()
        res2 = requests.get(url2).json()
        match_info_list.append([match_id,res1,res2])
        return match_info_list

In [8]:
# def get_rawdata(tier):

In [9]:
url = f'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={riot_api_key}'

In [10]:
res = requests.get(url).json()

In [11]:
df_creates=[]
for info in tqdm(res['entries'][:5]):
    summoner_id = info['summonerId']
    puu_id = get_puu_id(summoner_id,riot_api_key)
    match_ids = get_match_ids(puu_id,riot_api_key)
    match_info_list = get_matches_timelines(match_ids,riot_api_key)
    df_creates.extend(match_info_list)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.79s/it]


In [12]:
df = pd.DataFrame(df_creates,columns = ['match_id','matches','timelines'])
df

,match_id,matches,timelines
0,KR_7057884618,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_7063470458,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_7062919053,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_7057314789,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_7059309418,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


In [18]:
df.matches[1]['info']

{'endOfGameResult': 'GameComplete',
 'gameCreation': 1715278261679,
 'gameDuration': 1152,
 'gameEndTimestamp': 1715279444739,
 'gameId': 7063470458,
 'gameMode': 'CLASSIC',
 'gameName': 'teambuilder-match-7063470458',
 'gameStartTimestamp': 1715278292314,
 'gameType': 'MATCHED_GAME',
 'gameVersion': '14.9.580.2108',
 'mapId': 11,
 'participants': [{'allInPings': 0,
   'assistMePings': 1,
   'assists': 2,
   'baronKills': 0,
   'basicPings': 0,
   'bountyLevel': 0,
   'challenges': {'12AssistStreakCount': 0,
    'abilityUses': 185,
    'acesBefore15Minutes': 0,
    'alliedJungleMonsterKills': 1,
    'baronTakedowns': 0,
    'blastConeOppositeOpponentCount': 0,
    'bountyGold': 0,
    'buffsStolen': 0,
    'completeSupportQuestInTime': 0,
    'controlWardsPlaced': 0,
    'damagePerMinute': 1151.7576316589707,
    'damageTakenOnTeamPercentage': 0.3131007729207886,
    'dancedWithRiftHerald': 0,
    'deathsByEnemyChamps': 3,
    'dodgeSkillShotsSmallWindow': 0,
    'doubleAces': 0,
    '

In [23]:
df.timelines[0]

{'metadata': {'dataVersion': '2',
  'matchId': 'KR_7062473107',
  'participants': ['BHPce-2J5jqx5H608wA_h9ritxAVAx8xBWRQNGHn_tDv1E8uBAAKw-E8ZYPtHMes-KVFUqAPO-zEUg',
   'ggHq0Z2OMf-TBZd7GbMg02sC_CfqieWk2feR7Cy00wm0M_Uod_Cd_IQDE4IbdA_3wLuCrzisZOEXjw',
   'SeZ9KZUopVd1rAovGiIiyz61mRXUtJ3dnGMqHyHUQJmaHLXuI4SAsrB9W1MVYFcHCeIAIsx1fKKYWQ',
   'Z6UxhLfNi-_z7D67NjxiukTzpbE2yurerzSH2rTklC2NHJSY7ka6p0R7AhbxnQxx91qJqpkg2O4SZQ',
   'fyiy7jTWE7omL8qXVJNcicVo2lK_yBQv-lo4m93yAqPJ7FvtM8V7p3LJdtFLS4BbVapnTqg_hs5vwQ',
   'SdB7kUy5Od16BboN5ONGrWggV7yxGilNxGQWSarQS5I8b3COxvQ90CBeuHpSsr0Yt2LuegXhwCQKJQ',
   'UElSksorgQ7QEKml__UyC4dUdk4eEom0ZuTDjR1DhvSAmbddgwL_SNNoiwK8TEtlNQg56O-fYafRyg',
   'zAdwX-KaQ39J1P6riHl6vCDYXMQ-IHkWsrB5b_u659IOHQGiFgpUQeTL7pKShaxMpAEZ5e853Eq4zg',
   'laQdqsVnjaEIAnX2FMMiExitAL7CubxaQFzdnhZVttj7-AvZECBX8h2k7WIP8R6-SoLa9-TS_5NfGg',
   'dIOY5WU5aV_z5uGYJTuXPQVjC_vaVoqlPwBfZN68Bba4gAu4V4gKWdYCghXQ9PTWg3N3TcK6Ov9tRw']},
 'info': {'endOfGameResult': 'GameComplete',
  'frameInterval': 6000